# 文本分类实例

## Step1 导入相关包

In [10]:
import os
os.environ['HTTPs_proxy'] = 'http://127.0.0.1:7890'
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [12]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

In [13]:
import pandas as pd

train_data = dataset.to_pandas()
print(train_data)

      label                                             review
0         1  距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较...
1         1                       商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2         1         早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3         1  宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4         1               CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风
...     ...                                                ...
7760      0  尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7761      0  盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7762      0  看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7763      0  我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...
7764      0  说实在的我很失望，之前看了其他人的点评后觉得还可以才去的，结果让我们大跌眼镜。我想这家酒店以...

[7765 rows x 2 columns]


## Step3 划分数据集

In [14]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [15]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

In [16]:
from huggingface_hub import snapshot_download

In [17]:
# 通过huggingface_hub 的 snapshot_download()下载模型还是挂了
snapshot_download(repo_id="hfl/chinese-macbert-large", ignore_patterns=["*.h5", "*.ot", "*.msgpack"])

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

d:\anaconda\envs\roboflow\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sjf19\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


'C:\\Users\\sjf19\\.cache\\huggingface\\hub\\models--hfl--chinese-macbert-large\\snapshots\\1cf2677c782975600ce58e2961656b1b29eddbae'

In [18]:
import torch

# 目前只能手动
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

# 把数据集做切次然后转换成模型的输入
def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 创建模型

In [19]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")

Some weights of the model checkpoint at hfl/chinese-macbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

## Step6 创建评估函数

In [20]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

In [21]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [22]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=1,   # 训练时的batch_size
                               gradient_accumulation_steps=32,  # *** 梯度累加 *** 就是acc
                               gradient_checkpointing=True,     # *** 梯度检查点 *** 开启checkpointing重计算
                               optim="adafactor",               # *** adafactor优化器 *** 不使用adamW 优化器
                               per_device_eval_batch_size=1,    # 验证时的batch_size
                               num_train_epochs=1,              # 训练轮数
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=32,
gradient_checkpointing=True,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_toke

## Step8 创建Trainer

Trainer的data_collator和torch.utils.data.DataLoader的collate_fn的差异就是，**输出格式！**`torch.utils.data.DataLoader的collate_fn`的输出可以是各种格式，但`Trainer的data_collator`的输出只能是一个dict，这个dict的键必须包含“input_ids”，“attention_mask”等transformers models正常运算必要的参数的名称，如果需要，也可以加入任何transformers model.forward()可接受的参数名，而这些键对应的值也必须是transformers model中该键应该对应的输入值。
如果想让模型自动训练loss，还要在这个dict中加入如下键值对：{“labels”: labels in tensor type}，这样模型的输出里就有loss了。

In [23]:
from transformers import DataCollatorWithPadding

# *** 参数冻结 *** 只训练后面的全连接层
for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [24]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jinfeng-sun. Use `wandb login --relogin` to force relogin


  0%|          | 0/218 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
d:\anaconda\envs\roboflow\lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


{'loss': 0.6522, 'learning_rate': 1.9082568807339454e-05, 'epoch': 0.05}
{'loss': 0.6564, 'learning_rate': 1.81651376146789e-05, 'epoch': 0.09}
{'loss': 0.6442, 'learning_rate': 1.724770642201835e-05, 'epoch': 0.14}
{'loss': 0.6384, 'learning_rate': 1.63302752293578e-05, 'epoch': 0.18}
{'loss': 0.6341, 'learning_rate': 1.541284403669725e-05, 'epoch': 0.23}
{'loss': 0.6306, 'learning_rate': 1.4495412844036698e-05, 'epoch': 0.27}
{'loss': 0.6536, 'learning_rate': 1.3577981651376149e-05, 'epoch': 0.32}
{'loss': 0.6606, 'learning_rate': 1.2660550458715597e-05, 'epoch': 0.37}
{'loss': 0.6313, 'learning_rate': 1.1743119266055047e-05, 'epoch': 0.41}
{'loss': 0.6214, 'learning_rate': 1.0825688073394496e-05, 'epoch': 0.46}
{'loss': 0.6073, 'learning_rate': 9.908256880733946e-06, 'epoch': 0.5}
{'loss': 0.6219, 'learning_rate': 8.990825688073395e-06, 'epoch': 0.55}
{'loss': 0.6198, 'learning_rate': 8.073394495412845e-06, 'epoch': 0.6}
{'loss': 0.643, 'learning_rate': 7.155963302752295e-06, 'epoch

  0%|          | 0/777 [00:00<?, ?it/s]

{'eval_loss': 0.623303234577179, 'eval_accuracy': 0.6885456885456885, 'eval_f1': 0.8155487804878049, 'eval_runtime': 13.9758, 'eval_samples_per_second': 55.596, 'eval_steps_per_second': 55.596, 'epoch': 1.0}
{'train_runtime': 281.0381, 'train_samples_per_second': 24.865, 'train_steps_per_second': 0.776, 'train_loss': 0.6314176397586088, 'epoch': 1.0}


TrainOutput(global_step=218, training_loss=0.6314176397586088, metrics={'train_runtime': 281.0381, 'train_samples_per_second': 24.865, 'train_steps_per_second': 0.776, 'train_loss': 0.6314176397586088, 'epoch': 1.0})

In [25]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/777 [00:00<?, ?it/s]

{'eval_loss': 0.623303234577179,
 'eval_accuracy': 0.6885456885456885,
 'eval_f1': 0.8155487804878049,
 'eval_runtime': 13.8391,
 'eval_samples_per_second': 56.145,
 'eval_steps_per_second': 56.145,
 'epoch': 1.0}

In [26]:
trainer.predict(tokenized_datasets["test"])

  0%|          | 0/777 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.53457934,  0.05543418],
       [-0.4656182 ,  0.09469027],
       [-0.5132886 ,  0.18788995],
       ...,
       [-0.44224152,  0.17639095],
       [-0.14693949,  0.33757675],
       [-0.5578738 ,  0.13632208]], dtype=float32), label_ids=array([1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1

## Step10 模型预测

In [27]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [28]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [30]:
# xformers 只能在torch2.0用


In [29]:
pipe(sen)

[{'label': '好评！', 'score': 0.6683060526847839}]